In [1]:
import os
os.chdir('..')

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [24]:
from src.api.common.dependency_container import DependencyContainer
from urllib.parse import quote  

DC = DependencyContainer(); DC.initialize()
pregunta = '[Ya terminé el curso anterior. No aparece mi certificado. Qué hago?'
request = {'pregunta': pregunta, 'FAQs': faqs + faqs_unir}
response = DependencyContainer.get_faqs_workflow().execute(request)
faq = max(response['FAQs'], key=lambda x: float(x['Score']))  
if float(faq['Score']) > 6: content = f"<h2>{faq['FAQ']}</h2>{faq['Contenido']}"  
else: content = f'<h2>No encontramos ninguna FAQ relacionada a la pregunta. Dirígete al siguiente número de Whatsapp: <a href="https://web.whatsapp.com/send?phone=34689909323&text={quote(pregunta)}">contacta con un asesor.</a></h2>'
response['FAQs']

[{'FAQ': '4. No me ha llegado el certificado',
  'Contenido': '<div><p>El certificado no se envía, así que es mejor no esperarlo. Hay que tener en cuenta:</p><ol>\n<li>El certificado se descarga, pero sólo cuando se han terminado "todos los módulos" del curso. No hay un certificado por módulo.</li>\n<li>Al acabar el último módulo, verás que tienes acceso a otro que solo se utiliza para realizar una encuesta y descargar el certificado.</li>\n</ol></div>',
  'Razonamiento': 'Esta FAQ es relevante ya que proporciona información sobre la descarga del certificado al finalizar los módulos del curso, lo cual es pertinente para la pregunta dada.',
  'Score': '9'},
 {'FAQ': '3. He acabado el módulo y no me permite pasar al siguiente',
  'Contenido': '<div><p>Cada vez que se finaliza una actividad, el sistema tarda un poco (un par de minutos) en actualizarse y reconocer el avance. Lo mismo ocurre con la finalización del módulo. Si han transcurrido unos minutos después de haberlo terminado todo y